# Workflow Lesson -- Landsat Data, Loops and Functions




In [ ]:
# Import libraries
import warnings
from glob import glob
import os

import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
from shapely.geometry import box
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep

# Get Landsat data
et.data.get_data(url="https://ndownloader.figshare.com/files/21941085")

os.chdir(os.path.join(et.io.HOME,
                      'earth-analytics',
                      'data'))
warnings.simplefilter('ignore')


# This will make rioxarray run faster
rioxarray_option = rxr.set_options(export_grid_mapping=False)

##  Versions  of Tools

In order for this to run,
you need to have the  following  versions of tools installed.
Run the line below to see double check package versions.

```
rioxarray (0.3.0) deps:
  rasterio: 1.2.0
    xarray: 0.16.2
      GDAL: 3.1.4
```

In [ ]:
# You  should have rioxarray >= 0.3.0  and rasterio >= 1.2.0 to run   this
rxr.show_versions()

# Start Your Workflow  With Pseudocode

Coding can seem daunting when you are starting from scratch. Start with what 
you know - English or whatever your native language is! Write out the steps 
as comments FIRST. Then go back and fill out the code. 

This process is similar to starting a paper with an outline.

In this case, you know that you have two directories of landsat scenes to 
process. You could process each directory individually, howerever the code
to do this will get complex and difficult to manage quickly. 

Using loops and well-named functions you can simplify your workflow. But, 
to create useful workflows you need to first create a workflow design using 
pseudo code. 

Note some people will even diagram out their workflow.

## Save Intermediate Objects

The approach in this course so far to save objects has been to name objects
individually and use them as needed. However, **Python** has the `dictionary` 
structure that allows you to create unique keys that can be used to 
access stored objects. 

For example:

`modis["date-here"]["ndvi"]`

could be used to store the NDVI output layer for modis data at a particular date.

Below you will build out a workflow from scratch. You will start with 
the core loop as an individual item that loops through directories and 
parses important metadata (such as the date associated with a remote sensing
file).


## The Steps

1. To begin you have two scenes to process. This could warrant a LOOP to process both scenes

```
# 1. Loop through both landsat scenes
    * create a list with paths for both scenes
```

Now, you know from your previous assignment that the DATE for which the Landsat
scene  was  captured  and processed is stored in the filename itself.
Expand this pseudocode to add grabbing the scene date.

```
# 1. Loop through both landsat scenes
    * create a list with paths to both scenes
    * in each iteraction, grab the date from the directory name
    * create an empty dictionary with a key for that date
    
```

Below this pseudocode is implemented in a cell using comments. Add the code 
required to build out the pseudo-code into functioning code.

Build your loop iteratively and run it as you add a line or two of code to 
ensure things work as you expect them to. Notice below there are print 
statements added to ensure the output is as expected! 

This is a great way to ensure that your paths are correct.


## Build out the Full Workflow in Pseudocode

**fill in steps as you go**

Continue with this pseudocode approach and build out the entire workflow. 
In your first pass you won't get all of the individual steps down. This is ok
just get things started and build it out as you go. 

NOTE: this notebook as a demo has many cells, you'd most likely build out your 
workflow in a single cell or two. i generally like to add a loop AFTER the cell
that contains the functions that the loop calls.
```
# To begin create your pseudo code

# High level

# 1. Loop through each landsat directory of tif files (each scene)
#    * create a list of paths to directories for both scenes
#    * in each loop iteration, grab the date from the directory name
#    * create an empty dictionary with a key for that date
# 2. Create a list of all tif files that you will need in the scene's directory
# 3. Open / crop / clean the tif files that  you need for your analysis
# 4. Optional: combine into a single object?
# 5. Calculate veg indices
#    - NDVI
#    - NBR / dNBR
# 6. Landsat  Data Only: Apply cloud  mask  to  final  NBR /  NDVI  layers

```

# Once Your  Pseudocode  Is  Setup, Build Out Your Loop

You can start with the processing code OR with the loop. There is no 
right or wrong approach. The important part of this is to think about
the ENTIRE workflow first and then implement code associated with the 
workflow requirements. 


In [ ]:
# 1. Loop through each landsat directory of tif files (each scene)
#    * create a list of paths to directories for both scenes
#    * in each loop iteration, grab the date from the directory name
#    * create an empty dictionary with a key for that date

In [ ]:
# 1. Loop through each landsat directory of tif files (each scene)

# * create a list of paths to directories for both scenes
data_path = os.path.join("earthpy-downloads", "landsat-coldsprings-hw")
# view subdirectories of data
all_dirs = glob(os.path.join(data_path, "*"))
# * in each loop iteration, grab the date from the directory name
# * create an empty dictionary with a key for that date
for landsat_dir in all_dirs:
    all_files = []
    #  Use print statements as you begin your loop / workflow  design  as a check
    print(landsat_dir)
# 2. Create a list of all tif files that you will need in the scene's directory
    all_files = sorted(glob(os.path.join(landsat_dir, "*.tif")))

all_files

You now have the start of a loop. 

At this point you may want to ask yourself - do I really need ALL of the bands?

The  more data that you  process, the more  resources you  will use.  Always 
ask yourself 

1. What data do I need? 
2. How can I efficienctly make that workflow?

Below, you add the second element of the loop which 
involves grabbing the date for the landsat scene.


In [ ]:
# 1. Loop through both landsat scenes

#    * create a list with paths to both scenes
data_path = os.path.join("earthpy-downloads", "landsat-coldsprings-hw")
# view subdirectories of data
all_dirs = glob(os.path.join(data_path, "*"))

for landsat_dir in all_dirs:
    all_files = []
    print(landsat_dir)
    # Get all tif files in the directory
    all_files = sorted(glob(os.path.join(landsat_dir, "*.tif")))

    print("path: ", landsat_dir)
    # Get the landsat scene name / directory

    #    * in each iteraction, grab the date from the directory name
    landsat_scene = os.path.basename(os.path.normpath(landsat_dir))
    print("scene name: ", landsat_scene)

    # Get the date from the scene name
    print("date: ", landsat_scene[10:18])

#    * create an empty dictionary with a key for that date

all_files

Finally, add the dictionary to store your output data in.
Note there are s everal  ways to do this -  this is  one  way

In [ ]:
# Create empty dictionary
landsat_data = {}

# 1. Loop through both landsat scenes

#    * create a list with paths to both scenes
data_path = os.path.join("earthpy-downloads",
                         "landsat-coldsprings-hw")
# view subdirectories of data
all_dirs = glob(os.path.join(data_path, "*/"))

for landsat_dir in all_dirs:
    all_files = []
    print(landsat_dir)
    # Get all tif files in the directory
    all_files = sorted(glob(os.path.join(landsat_dir, "*.tif")))

    print("path: ", landsat_dir)
    # Get the landsat scene name / directory

    #    * in each iteraction, grab the date from the directory name
    landsat_scene = os.path.basename(os.path.normpath(landsat_dir))
    print("scene name: ", landsat_scene)

    # Get the date from the scene name
    date = landsat_scene[10:18]
    landsat_data[date] = []

    print("date: ", date)
    print("\n")

#    * create an empty dictionary with a key for that date

landsat_data

###  Take  A  Break -  Clean  Things Up

Everything is printing nicely above so you can clean up your code and remove 
print statements now. 

This will be the core workflow. You are now ready to add the remote sensing 
processing steps to the loop. 

**Note** - if you have a big loop, you may want to 
only process two directories at a time rather than parsing through the entire
loop each time you run the cell. 



###  Revisit Your PseudoCode
Remember that this is an iterative process.


In [ ]:
# 1. Loop through each landsat directory of tif files (each scene)
#    * create a list of paths to directories for both scenes
#    * in each loop iteration, grab the date from the directory name
#    * create an empty dictionary with a key for that date
# 2. Create a list of all tif files that you will need in the scene's directory (TODO  Which  ones  do you need?)
# 3. Open / crop / clean the tif files that  you need for your analysis
#     *  create a loop to process each tif  file
#     *  open crop extent and open  / crop data (this may be one or  more steps)

# 4. Optional: combine into a single object?
# 5. Calculate veg indices
#    - NDVI
#    - NBR / dNBR
# 6. Landsat  Data Only: Apply cloud  mask  to  final  NBR /  NDVI  layers

In [ ]:
# Open fire boundary
fire_boundary_path = os.path.join("cold-springs-fire",
                                  "vector_layers",
                                  "fire-boundary-geomac",
                                  "co_cold_springs_20160711_2200_dd83.shp")
fire_boundary = gpd.read_file(fire_boundary_path)

In [ ]:
# Create empty dictionary
landsat_data = {}

# 1. Loop through both landsat scenes

#    * create a list with paths to both scenes
data_path = os.path.join("earthpy-downloads",
                         "landsat-coldsprings-hw")
# view subdirectories of data
all_dirs = glob(os.path.join(data_path, "*/"))


for a_dir in all_dirs:
    # Get the landsat scene name / directory & extract date
    landsat_scene = os.path.basename(os.path.normpath(landsat_dir))

    # Get the date from the scene name
    date = landsat_scene[10:18]

    # 2. Create a list of all tif files in the scene's directory
    # Get JUST THE tif files THAT YOU NEED  in the directory
    all_files = sorted(glob(os.path.join(a_dir, "*band[3-7]*.tif")))
    landsat_data[date] = all_files
    # 3. Open / crop the files
    # requirements: crop extent - you need a boundary - add it to the code)
    # 4. Combine the cropped files - in this case you will want to use many different bands for the 2 veg indices
    # 5. Clean the spectral data
    # 6. Calculate veg indices
    #    - NDVI
    #    - NBR / dNBR
    # 7.  mask final veg index data

landsat_data

## Continue to Iteratively  Add Code to Your PseudoCode

As you  can  see  above, you can iteratively add code to your pseudocode. Above 
I  created a new  celle ach time  only  for   teaching  purposes. You may just  
buildout your workflow in a few cells.

Take it step by step and think about potential issues that you may encounter in each step.

Developing the example below was a process:

1. Create your pseudo code for the steps that you want to perform 
2. Create your EMPTY outer loop as a test to implement this across multiple directories (above)
3. Begin to add code below each line of pseudo code. as you add your code:
    1. Consider the inputs and ouputs of each function (use help(function_name) to help with this. 
    2. Consider where things could fail and add tests using try: statements or conditionals (`if` statements)
    3.  Use `print()` statements in your loops to keep track of what the loop is doing
 

In [ ]:
# Create empty dictionary
landsat_data = {}

# 1. Loop through both landsat scenes

#    * create a list with paths to both scenes
data_path = os.path.join("earthpy-downloads",
                         "landsat-coldsprings-hw")
# view subdirectories of data
all_dirs = glob(os.path.join(data_path, "*/"))


for landsat_dir in all_dirs:
    # Get the landsat scene name / directory & extract date
    landsat_scene = os.path.basename(os.path.normpath(landsat_dir))
    print("Now processing: ", a_dir)
    # Get the date from the scene name
    date = landsat_scene[10:18]

    # 2. Create a list of all tif files in the scene's directory
    # Get JUST THE tif files THAT YOU NEED  in the directory
    all_bands = sorted(glob(os.path.join(a_dir, "*band[3-7]*.tif")))

    # 3. Open / crop the files
    #     *  create a loop to process each tif  file
    all_bands_list = []
    for aband in all_bands:
        # print(aband)
        all_bands_list.append(aband)

    # You  haven't opened any  data  yet but have developed a nested loop
    # workflow that  grabs  the  data that you need to process
    landsat_data[date] = all_bands_list
    # requirements: crop extent - you need a boundary - add it to the code)
    # 4. Combine the cropped files - in this case you will want to use many different bands for the 2 veg indices
    # 5. Clean the spectral data
    # 6. Calculate veg indices
    #    - NDVI
    #    - NBR / dNBR
    # 7.  mask final veg index data

landsat_data

In [ ]:
# Open the data in  the loop
data_path = os.path.join("earthpy-downloads",
                         "landsat-coldsprings-hw")

# Create empty dictionary
landsat_data = {}

# First create the high level loop to loop through each landsat directory of tif files (each scene)

# Get all directories  - remember that getting a list makes things more scalabe
all_dirs = glob(os.path.join(data_path, "*"))


# Create a loop variable that you can use to run your code for a single directory
for a_dir in all_dirs:
    # Get the landsat scene name / directory & extract date
    landsat_scene = os.path.basename(os.path.normpath(a_dir))
    print("Now processing: ", a_dir)
    # Get the date from the scene name
    date = landsat_scene[10:18]

    # Get all tif files in the directory
    all_bands = sorted(glob(os.path.join(a_dir, "*band[3-7]*.tif")))

    all_bands_list = []
    for aband in all_bands:

        crop_bound_box = [box(*fire_boundary.bounds.loc[0])]
        # Open and clip a band to the fire_boundary extent
        band = rxr.open_rasterio(aband,
                                 masked=True).rio.clip(crop_bound_box,
                                                       crs=fire_boundary.crs,
                                                       all_touched=True,
                                                       from_disk=True).squeeze()
        all_bands_list.append(band)

    landsat_data[date] = all_bands_list

landsat_data.keys()

In [ ]:
landsat_data['20160621'][0]

##  Functions Make Your Code Easier to Read

Remember that functions:

1. Make your code easier to read and more expressive  if named well
2. Create  variables in a temporary environment to save memory
3. Can be reused / make code: DRY

Are you are developing your workflow, think about steps that are 
repeated over and over. Creating functions to do things like "open and clean  data"
can be a good way to make your code more efficient and DRY.

The function below is in your homework notebook. You will need to clean up
the docstring as a part of your homework.

In [ ]:
def open_clean_bands(band_path,
                     crop_bound,
                     valid_range=None,
                     variable=None):
    """Open and clean a single landsat band .
    FIX THIS FUNCTION DOCSTRING!!

    Parameters
    -----------
    band_path:string
    A path to the array to be opened
    crop_bound:geopandas GeoDataFrame
    A geopandas dataframe to be used to crop the raster data using rioxarray clip().
    valid_range:tuple (optional)
     A tuple of min and max range of values for the data. Default = None


    """

    crop_bound_box = [box(*crop_bound.bounds.loc[0])]

    try:
        band = rxr.open_rasterio(band_path,
                                 masked=True).rio.clip(crop_bound_box,
                                                       crs=crop_bound.crs,
                                                       all_touched=True,
                                                       from_disk=True).squeeze()
    except:
        print("Oops - I couldn't clip your data. This may be due to a crs error.")

    # Only mask the data to the valid range if a valid range tuple is provided
    if valid_range is not None:
        mask = ((band < valid_range[0]) | (band > valid_range[1]))
        band = band.where(~xr.where(mask, True, False))

    return band

In [ ]:
# Apply the function to your workflow
data_path = os.path.join("earthpy-downloads",
                         "landsat-coldsprings-hw")

# Create empty dictionary
landsat_data = {}

# First create the high level loop to loop through each landsat directory of tif files (each scene)

# Get all directories  - remember that getting a list makes things more scalabe
all_dirs = glob(os.path.join(data_path, "*"))


# Create a loop variable that you can use to run your code for a single directory
for a_dir in all_dirs:
    # Get the landsat scene name / directory & extract date
    landsat_scene = os.path.basename(os.path.normpath(a_dir))
    print("Now processing: ", a_dir)
    # Get the date from the scene name
    date = landsat_scene[10:18]

    # Get all tif files in the directory
    all_bands = sorted(glob(os.path.join(a_dir, "*band[3-7]*.tif")))

    all_bands_list = []
    for aband in all_bands:
        # print(aband)
        all_bands_list.append(open_clean_bands(aband,
                                               crop_bound=fire_boundary))

    landsat_data[date] = all_bands_list

landsat_data.keys()

In [ ]:
#  Eventually  you will store a stacked  dataset here
landsat_data['20160621']

In [ ]:
landsat_data.keys()

### Open Each Band  in  Your Loop

Once you have setup your code and it works on a scene, or within your loop you 
can think about making it modular. I often look at my code and find steps that 
are related that might pair nicely together in a function. Well-named Functions will 
make your code easier to read and trouble shoot.

In [ ]:
data_path = os.path.join("earthpy-downloads",
                         "landsat-coldsprings-hw")

# Get all directories - remember that getting a list makes things more scalable
all_dirs = glob(os.path.join(data_path, "*"))

landsat_dict = {}
# Create a loop variable that you can use to run your code for a single directory
for a_dir in all_dirs:
    landsat_scene = os.path.basename(os.path.normpath(a_dir))
    scene_date = landsat_scene[10:18]
    print("Now processing: ", scene_date)

    # Get all tif files in the directory
    all_bands = sorted(glob(os.path.join(a_dir,
                                         "*band[3-7]*.tif")))

    all_bands_list = []
    for aband in all_bands:
        band_crop = open_clean_bands(aband,
                                     crop_bound=fire_boundary,
                                     valid_range=(0, 10000))
        all_bands_list.append(band_crop)

    landsat_dict[scene_date] = all_bands_list

landsat_dict

## Combine A List of  Xarray  Objects With  Xarray.concat

you  can  use xarray `concat`  to combine a list of xarray objects.
This will e ffectively "stack" all of  your  raster data  into  one 
single data  cube rather than  a list of objects.

`xr.concat(landsat_arrays, dim="band")`

In [ ]:
# list of xarray objects
all_bands_list

In [ ]:
# combine  the  list  into a single xarray object for  simpler processin
xr.concat(all_bands_list, dim="band")

Now, add that step to your workflow.

In [ ]:
data_path = os.path.join("earthpy-downloads",
                         "landsat-coldsprings-hw")

# Get all directories - remember that getting a list makes things more scalable
all_dirs = glob(os.path.join(data_path, "*"))

landsat_dict = {}
# Create a loop variable that you can use to run your code for a single directory
for a_dir in all_dirs:
    landsat_scene = os.path.basename(os.path.normpath(a_dir))
    scene_date = landsat_scene[10:18]
    print("Now processing: ", scene_date)

    # Get all tif files in the directory
    all_bands = sorted(glob(os.path.join(a_dir,
                                         "*band[3-7]*.tif")))

    all_bands_list = []
    for aband in all_bands:
        band_crop = open_clean_bands(aband,
                                     crop_bound=fire_boundary,
                                     valid_range=(0, 10000))
        all_bands_list.append(band_crop)
    # Combine all xarray  objects in the list above into a single array.
    all_bands_xr = xr.concat(all_bands_list,  dim="band")

    # Add that combined object to  your  dictionary
    landsat_dict[scene_date] = all_bands_xr

landsat_dict

# Summary - Workflow Tips   

After creating the functions notice how much easier the code is to read!
At this point you have a working loop and some of the pieces that you
need to complete your workflow. 

Functions are a nice way to organize your work because they allow you to 
modularize your workflow, extracting steps into individual components that you 
can test. 

## Dictionaries  Review

This section just reviews dictionaries in case you want to spend some more time  
understanding how they work.

You may want to calculate your vegetation indices using functions or steps.
Below two additional objects are created for ndvi and nbr respectively. Note that 
you could combine these into one dict or keep them separate depending upon 
your workflow! Or maybe you decided that you do'nt need the dict of cleaned 
data. you ONLY want the dict for NDVI and NBR

In [ ]:
# Nested dictionary can be used to store information hierarchically as an option

veg_indices = {}

veg_indices["date"] = {"ndvi": .8, "nbr": .8}

veg_indices["date"]["ndvi"]

# IGNORE THIS FOR THIS YEAR -- Cloud Masks for Landsat Data 


## Open the Pixel QA layer For Cloud Masking

Once you have the above figured out, you can open the  pixel qa layer
using the same function you  used  to open  the  other  bands.
For this layer, you don't need to worry about a valid range of  values  
like you do for the bands.

If you recall from the previous in class notebooks, it is fastest to 
apply the cloud mask to your final processed NDVI / NBR layers rather  
than masking each band. 

In [ ]:
# Example of opening the pixel qa layer which can be used to mask your final
#  veg  index data
pixel_qa_path = glob(os.path.join(a_dir,
                                  "*pixel_qa*.tif"))[0]
pixel_qa = open_clean_bands(pixel_qa_path,
                            crop_bound=fire_boundary)

pixel_qa

##  We  Are Skipping Masking  This Year -  So  You  Can  Ignore this

The mask layer above is used to mask clouds. You can pass the array and values
that you wish to mask to the function

In [ ]:
# Example mask function
def mask_data(xr_array,
              pixel_qa,
              mask_values):
    """
    Mask the data

    xr_array : xarray DataArray  object to be masked

    pixel_qa : xarray DataArray object
        An xarray  DataArray  object  containing a pixel qa layer
    mask_values : list
        A list of values needed to create the cloud mask
    """

    if len(pixel_qa.shape) == 3 & pixel_qa.shape[0] == 1:
        pixel_qa = pixel_qa.squeeze()

    return xr_array.where(~pixel_qa.isin(mask_values))

In [ ]:
aband

In [ ]:
vals = [328, 392, 840, 904, 1350, 352, 368,
        416, 432, 480, 864, 880, 928, 944, 992, 480, 992]

#  Mask a Single xarray object  -  pretend  this is NDVI  data
a_band = open_clean_bands(aband,
                          crop_bound=fire_boundary,
                          valid_range=(0, 10000))
#  Mask a single layer using a mask function
mask_data(xr_array=a_band,
          pixel_qa=pixel_qa,
          mask_values=vals)